In [1]:
import pandas as pd
import onnxruntime as rt

from group2.partition_tests import (
    partition_test_binary,
    partition_test_numeric,
)
from group2.metamorphic_tests import (
    metamorphic_test,
    neutralize_opm_judgements,
    normalize_documentation_intensity,
)

In [2]:
df = pd.read_csv("../data/test.csv")
X = df.drop(columns=["checked"], errors="ignore")
y = df["checked"]

In [3]:
class ONNXModelWrapper:
    def __init__(self, model_path):
        self.session = rt.InferenceSession(model_path, providers=["CPUExecutionProvider"])
        self.input_name = self.session.get_inputs()[0].name
        self.output_name = self.session.get_outputs()[0].name

    def predict(self, X):
        preds = self.session.run(
            [self.output_name],
            {self.input_name: X.values.astype("float32")}
        )[0]
        return preds.astype(int)

In [4]:
# Load Group 1 models
model_1 = ONNXModelWrapper("../group1/model_1.onnx")
model_2 = ONNXModelWrapper("../group1/model_2.onnx")

In [5]:
from group2.train_models import toxic_features, safe_core

print("=== MODEL 1: PARTITION TESTS ===")

for feature in toxic_features:
    if feature not in X.columns:
        continue
    if X[feature].dropna().nunique() <= 2:
        partition_test_binary(model_1, X, feature)
    else:
        partition_test_numeric(model_1, X, feature)

print("\n=== MODEL 1: METAMORPHIC TESTS ===")
metamorphic_test(model_1, X, neutralize_opm_judgements, name="Neutralize OPM Judgements", threshold=0.05)
metamorphic_test(model_1, X, normalize_documentation_intensity, name="Normalize documentation verbosity", threshold=0.05)

Saved model_1.onnx with full 317-column interface.
Saved model_2.onnx with full 317-column interface.
=== MODEL 1: PARTITION TESTS ===
[Partition] persoonlijke_eigenschappen_flexibiliteit_opm
  Group 0 rate: 0.052
  Group 1 rate: 0.059
  Absolute diff: 0.007
  Risk ratio: 1.14
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_doorzettingsvermogen_opm
  Group 0 rate: 0.053
  Group 1 rate: 0.058
  Absolute diff: 0.006
  Risk ratio: 1.11
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_motivatie_opm
  Group 0 rate: 0.052
  Group 1 rate: 0.058
  Absolute diff: 0.005
  Risk ratio: 1.10
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_houding_opm
  Group 0 rate: 0.052
  Group 1 rate: 0.058
  Absolute diff: 0.006
  Risk ratio: 1.12
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_uiterlijke_verzorging_opm
  Group 0 rate: 0.053
  Group 1 rate: 0.058
  Absolute diff: 0.005
  Risk ratio: 1.08
  RESULT: PASS
[Partition] afspraak_aantal_woorden
  Low (≤ median) rate:  0.059
  Hi

{'name': 'Normalize documentation verbosity',
 'changed': np.int64(0),
 'fraction_changed': np.float64(0.0),
 'passed': np.True_}

In [6]:
print("=== MODEL 2: PARTITION TESTS ===")

for feature in toxic_features:
    if feature not in X.columns:
        continue
    if X[feature].dropna().nunique() <= 2:
        partition_test_binary(model_2, X, feature)
    else:
        partition_test_numeric(model_2, X, feature)

print("\n=== MODEL 2: METAMORPHIC TESTS ===")
metamorphic_test(model_2, X, neutralize_opm_judgements, name="Neutralize OPM Judgements", threshold=0.05)
metamorphic_test(model_2, X, normalize_documentation_intensity, name="Normalize documentation verbosity", threshold=0.05)

=== MODEL 2: PARTITION TESTS ===
[Partition] persoonlijke_eigenschappen_flexibiliteit_opm
  Group 0 rate: 0.012
  Group 1 rate: 0.015
  Absolute diff: 0.003
  Risk ratio: 1.24
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_doorzettingsvermogen_opm
  Group 0 rate: 0.011
  Group 1 rate: 0.015
  Absolute diff: 0.004
  Risk ratio: 1.35
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_motivatie_opm
  Group 0 rate: 0.011
  Group 1 rate: 0.015
  Absolute diff: 0.003
  Risk ratio: 1.27
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_houding_opm
  Group 0 rate: 0.011
  Group 1 rate: 0.015
  Absolute diff: 0.004
  Risk ratio: 1.33
  RESULT: PASS
[Partition] persoonlijke_eigenschappen_uiterlijke_verzorging_opm
  Group 0 rate: 0.012
  Group 1 rate: 0.014
  Absolute diff: 0.002
  Risk ratio: 1.14
  RESULT: PASS
[Partition] afspraak_aantal_woorden
  Low (≤ median) rate:  0.012
  High (> median) rate: 0.015
  Absolute diff: 0.003
  Risk ratio: 1.30
  RESULT: PASS

=== MODEL 2: METAM

{'name': 'Normalize documentation verbosity',
 'changed': np.int64(0),
 'fraction_changed': np.float64(0.0),
 'passed': np.True_}

In [7]:
from group2.permutation_importance import permutation_importance_accuracy

print("\n=== MODEL 1: PERMUTATION IMPORTANCE ===")

imp_toxic_m1 = permutation_importance_accuracy(model_1, X, y, toxic_features)
imp_safe_m1 = permutation_importance_accuracy(model_1, X, y, safe_core)

print("Model 1 total toxic importance:", sum(imp_toxic_m1.values()))
print("Model 1 total safe importance :", sum(imp_safe_m1.values()))



=== MODEL 1: PERMUTATION IMPORTANCE ===
[Permutation importance – accuracy drop]
  baseline accuracy: 0.8602
  total importance over 6 features: 0.0000
  mean per-feature importance: 0.0000
[Permutation importance – accuracy drop]
  baseline accuracy: 0.8602
  total importance over 9 features: 0.0129
  mean per-feature importance: 0.0014
Model 1 total toxic importance: 0.0
Model 1 total safe importance : 0.012933333333333278


In [8]:
print("\n=== MODEL 2: PERMUTATION IMPORTANCE ===")

imp_toxic_m2 = permutation_importance_accuracy(model_2, X, y, toxic_features)
imp_safe_m2 = permutation_importance_accuracy(model_2, X, y, safe_core)

print("Model 2 total toxic importance:", sum(imp_toxic_m2.values()))
print("Model 2 total safe importance :", sum(imp_safe_m2.values()))


=== MODEL 2: PERMUTATION IMPORTANCE ===
[Permutation importance – accuracy drop]
  baseline accuracy: 0.8556
  total importance over 6 features: 0.0000
  mean per-feature importance: 0.0000
[Permutation importance – accuracy drop]
  baseline accuracy: 0.8556
  total importance over 9 features: 0.0027
  mean per-feature importance: 0.0003
Model 2 total toxic importance: 0.0
Model 2 total safe importance : 0.0027333333333333654
